# FAKE REVIEW NODE EMBEDDING

In [86]:
# importing all necessary libraries
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import os
import networkx as nx
from sklearn import preprocessing, model_selection
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
#from node2vec import Node2Vec as n2v
sns.set()

%matplotlib inline

In [87]:
import os, sys
from copy import deepcopy
import numpy as np
import pandas as pd

import torch as th
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

In [88]:
print(f"Torch version: {th.__version__}.  CUDA version: {th.version.cuda}")

Torch version: 1.12.1+cu113.  CUDA version: 11.3


In [89]:
dgl_version = f"dgl-cu{th.version.cuda.replace('.', '')}==0.7.2"
dgl_version

'dgl-cu113==0.7.2'

In [90]:
# Follow instructions at https://www.dgl.ai/pages/start.html
!{sys.executable} -m pip install {dgl_version} -f https://data.dgl.ai/wheels/repo.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html


In [91]:
os.environ['DGLBACKEND'] = 'pytorch'
import dgl
from dgl import function as fn

In [92]:
dgl.__version__

'0.7.2'

In [93]:
from dgl.dataloading.pytorch import NodeDataLoader
device = th.device('cuda' if th.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [94]:
import torch
import torch.nn as nn
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
import argparse

from dgl.sampling import node2vec_random_walk

In [95]:
#importing libraries
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import statistics
import math
import os
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
#nltk.download('all')
import re
pd.set_option('max_colwidth', 800)


#importing Libraies
import networkx as nx
import matplotlib.pyplot as plt
from numpy import genfromtxt
import collections
from collections import OrderedDict

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LinearRegression,LogisticRegression, RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [96]:
#loading dataset
df=pd.read_csv('/content/category 2.csv')
df.head()

,category,rating,label,text_
0,Kindle_Store_5,5.0,CG,This book I can't wait for more by this author. I will definitely be reading the next one
1,Kindle_Store_5,5.0,CG,Love this series cant wait for more by this author. I will keep my eyes peeled for the next
2,Kindle_Store_5,5.0,CG,Read the books 2 time reading. The characters are well developed and believable. The story flows smoothly.
3,Kindle_Store_5,4.0,CG,very enjoyable. I would recommend this book to anyone. I really enjoyed this book.
4,Kindle_Store_5,5.0,CG,Great story can't wait for more. The characters are well developed and the story is well written.


In [97]:
#df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [98]:
df.shape

(4730, 4)

In [99]:
df['label'].value_counts()

CG    2365
OR    2365
Name: label, dtype: int64

In [100]:
# dropping unnecessary columns. 
df.drop(['category', 'rating'], axis = 1, inplace=True)
df.head()

,label,text_
0,CG,This book I can't wait for more by this author. I will definitely be reading the next one
1,CG,Love this series cant wait for more by this author. I will keep my eyes peeled for the next
2,CG,Read the books 2 time reading. The characters are well developed and believable. The story flows smoothly.
3,CG,very enjoyable. I would recommend this book to anyone. I really enjoyed this book.
4,CG,Great story can't wait for more. The characters are well developed and the story is well written.


In [101]:
z = {'OR' : 1, 'CG' : 0}  # 1 for original reviews and 0 for computer generated reviews
df['label'] = df['label'].map(z)

In [102]:
# Cleaning Raw tweets
def clean_text(text):
    
    #remove emails
    text = ' '.join([i for i in text.split() if '@' not in i])
    
    #remove web address
    text = re.sub('http[s]?://\S+', '', text)

    #remove brackets and parantheses (and removing the text inside the brackets and parantheses)
    text = re.sub("[\(\[].*?[\)\]]", "", text)

    #remove punctuations
    text = re.sub(r'[^\w\s]', ' ', text)
    
    #Filter to allow only alphabets and numbers
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    #Remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    #remove double spaces 
    text = re.sub('\s+', ' ', text)
    
    return text

In [103]:
# applying that into function.
df["clean_text"] = df['text_'].apply(lambda x: clean_text(x))
df.head(10)

,label,text_,clean_text
0,0,This book I can't wait for more by this author. I will definitely be reading the next one,This book I can t wait for more by this author I will definitely be reading the next one
1,0,Love this series cant wait for more by this author. I will keep my eyes peeled for the next,Love this series cant wait for more by this author I will keep my eyes peeled for the next
2,0,Read the books 2 time reading. The characters are well developed and believable. The story flows smoothly.,Read the books time reading The characters are well developed and believable The story flows smoothly
3,0,very enjoyable. I would recommend this book to anyone. I really enjoyed this book.,very enjoyable I would recommend this book to anyone I really enjoyed this book
4,0,Great story can't wait for more. The characters are well developed and the story is well written.,Great story can t wait for more The characters are well developed and the story is well written
5,0,This was a very good book. The characters were well developed and kept my interest.,This was a very good book The characters were well developed and kept my interest
6,0,"Fun reading - lots of action and suspense. Not too much sex, just a little bit of",Fun reading lots of action and suspense Not too much sex just a little bit of
7,0,Absolutely love Cage! This series is an amazing story and you will not be disappointed.,Absolutely love Cage This series is an amazing story and you will not be disappointed
8,0,Elle Casey Thank You for reading this book. It is a great book for a beginner to the,Elle Casey Thank You for reading this book It is a great book for a beginner to the
9,0,Great for all thos ebeta readers. The story is a good read. If you are looking,Great for all thos ebeta readers The story is a good read If you are looking


In [104]:
# dropping the uncleaned text column "text"
df = df.drop(['text_'], axis=1)

In [105]:
# renaming the column
df.rename(columns = {'clean_text':'text', 'label': 'Class'}, inplace = True)
df.head()

,Class,text
0,0,This book I can t wait for more by this author I will definitely be reading the next one
1,0,Love this series cant wait for more by this author I will keep my eyes peeled for the next
2,0,Read the books time reading The characters are well developed and believable The story flows smoothly
3,0,very enjoyable I would recommend this book to anyone I really enjoyed this book
4,0,Great story can t wait for more The characters are well developed and the story is well written


In [106]:
import spacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words
print(sw_spacy)

{'’m', '‘d', 'why', 'off', 'this', 'one', 'enough', 'these', 'could', 'fifteen', 'yourself', 'ten', 'move', 'whom', 'seems', 'mostly', 'almost', 'when', 'its', 'rather', 'before', 'wherein', 'or', 'somewhere', 'therein', 'twenty', 'me', '’d', 'except', 'were', 'either', 'anyhow', 'most', 'whence', 'moreover', 'she', 'everywhere', 'her', 'throughout', 'alone', 'latterly', 'six', 'sometime', 'against', 'hers', 'top', 'after', 'as', 'above', 'besides', 'put', 'seemed', 'via', 'side', "n't", 'your', 'without', 'whatever', 'whose', 'ca', 'yours', '‘s', 'very', 'regarding', 'already', 'whither', 'is', 'wherever', 'fifty', 'others', 'more', 'behind', 'thru', 'was', 'now', 'where', 'few', 'along', 'once', 'thence', 'forty', 'he', 'whereby', '’s', '‘ve', 'it', 'three', 'and', 'get', 'would', 'both', 'own', 'hereupon', 'herself', 'made', 'us', 'else', 'if', 'around', 'seeming', 'used', '’ve', 'even', 'myself', 'n’t', 'an', 'otherwise', 'the', 'then', 'upon', 'something', 'than', 'least', 'only',

In [107]:
import nltk
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')
print(sw_nltk)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [108]:
sw_nltk.extend(['th','stay'])
print(len(sw_nltk))

181


In [109]:
# preprocessing the data (removing stopwords, convert the sentence to lower case, and also applying lemmatization to get meaningful words )
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
#stop_words = set(stopwords.words('english'))
ps=PorterStemmer()
wnl=WordNetLemmatizer()
def clean_lemma(review):
    review = review.lower()
    review = review.split()
    review = [wnl.lemmatize(word) for word in review if word not in sw_nltk]
    review = [wnl.lemmatize(word) for word in review if word not in sw_spacy]
    review = ' '.join(review)
    return review

In [110]:
# using lemmatization
df['text']=df['text'].apply(clean_lemma)
df.head()

,Class,text
0,0,book wait author definitely reading
1,0,love series cant wait author eye peeled
2,0,read book time reading character developed believable story flow smoothly
3,0,enjoyable recommend book enjoyed book
4,0,great story wait character developed story written


### FEATURE EXTRACTION

In [111]:
x = df['text']
y = df['Class']

In [112]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
cv=TfidfVectorizer()
corpii=cv.fit_transform(x) 

In [113]:
feature_names = cv.get_feature_names()
#feature_names

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [114]:
len(feature_names)

11355

In [115]:
u1 = pd.DataFrame(corpii.toarray(), columns=feature_names)
u1.head()

,aa,aack,aaliyah,aaron,ab,abandoned,abandoning,abandonment,abbey,abbie,...,zekes,zelda,zenina,zero,zimmerman,zoe,zoey,zombie,zone,zoo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### FEATURE SELECTION

In [116]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
# Two features with highest chi-squared statistics are selected
chi2_features = SelectKBest(chi2, k = 800)
X_kbest_features = chi2_features.fit_transform(u1, y)

In [117]:
chi_support = chi2_features.get_support()

In [118]:
chi_feature = u1.loc[:,chi_support].columns.tolist()

In [119]:
chi_feature

['able',
 'absolutely',
 'accept',
 'accident',
 'actually',
 'adam',
 'add',
 'added',
 'addition',
 'admit',
 'adored',
 'advice',
 'age',
 'agent',
 'ago',
 'air',
 'alex',
 'alive',
 'alot',
 'alpha',
 'amazingly',
 'amazon',
 'amish',
 'amy',
 'angel',
 'angst',
 'answer',
 'anxiously',
 'appearance',
 'appears',
 'appreciate',
 'arc',
 'ashley',
 'aspect',
 'assassin',
 'attention',
 'attraction',
 'audience',
 'author',
 'available',
 'balance',
 'ball',
 'band',
 'based',
 'basically',
 'battle',
 'beautifully',
 'beauty',
 'beginner',
 'beginning',
 'behavior',
 'believable',
 'believe',
 'bella',
 'belong',
 'better',
 'biker',
 'birthday',
 'bit',
 'black',
 'blake',
 'blood',
 'blown',
 'blurb',
 'book',
 'bos',
 'bought',
 'boyfriend',
 'brand',
 'brandon',
 'break',
 'breaking',
 'breath',
 'brie',
 'brief',
 'brilliant',
 'bring',
 'brother',
 'brought',
 'build',
 'built',
 'bundle',
 'butt',
 'buy',
 'buying',
 'cabbage',
 'cade',
 'cake',
 'captivating',
 'captive',
 

In [120]:
X_kbest_features

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.05766398, 0.        , 0.        , ..., 0.05871438, 0.05493256,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.05037755,
        0.        ]])

In [121]:
u3 = pd.DataFrame(X_kbest_features, columns = chi_feature)
u3.head()

,able,absolutely,accept,accident,actually,adam,add,added,addition,admit,...,worked,working,worried,writing,written,wrong,wrote,year,young,younger
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.41397,0.0,0.0,0.0,0.0,0.0


In [122]:
# SOURCE : https://github.com/dmlc/dgl/issues/3364
src, dst = np.nonzero(X_kbest_features)

In [123]:
src

array([   0,    0,    0, ..., 4729, 4729, 4729])

In [124]:
dst

array([ 38,  64, 172, ..., 749, 795, 798])

In [125]:
g = dgl.graph((src, dst))

In [126]:
n_nodes = g.num_nodes()
n_nodes

4730

In [127]:
nodes = g.nodes()
nodes

tensor([   0,    1,    2,  ..., 4727, 4728, 4729])

In [128]:
n_nodes1 = g.number_of_nodes()
n_nodes1

4730

In [129]:
g.ndata['feat'] = torch.tensor(X_kbest_features)
g.ndata['feat'] 

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0577, 0.0000, 0.0000,  ..., 0.0587, 0.0549, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0504, 0.0000]],
       dtype=torch.float64)

In [130]:
n_train = int(n_nodes1 * 0.6)
n_val = int(n_nodes1 * 0.2)

In [131]:
train_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(train_mask)
train_mask

tensor([False, False, False,  ..., False, False, False])

In [132]:
val_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(val_mask)
val_mask

tensor([False, False, False,  ..., False, False, False])

In [133]:
test_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(test_mask)
test_mask

tensor([False, False, False,  ..., False, False, False])

In [134]:
train_mask[:n_train] = True

In [135]:
val_mask[n_train:n_train + n_val] = True

In [136]:
test_mask[n_train + n_val:] = True

In [137]:
g.ndata['train_mask'] = train_mask

In [138]:
g.ndata['val_mask'] = val_mask

In [139]:
g.ndata['test_mask'] = test_mask

In [140]:
s_array = df['Class'].to_numpy()
s_array

array([0, 0, 0, ..., 1, 0, 1])

In [141]:
t1 = torch.tensor(s_array)
t1

tensor([0, 0, 0,  ..., 1, 0, 1])

In [142]:
g.ndata["label"] =t1

In [143]:
label = g.ndata["label"]

In [144]:
label[train_mask]

tensor([0, 0, 0,  ..., 1, 1, 1])

In [145]:
label[val_mask]

tensor([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,

In [146]:
label[test_mask]

tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
        1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
        1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
        1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
        1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1,

In [147]:
train_mask

tensor([ True,  True,  True,  ..., False, False, False])

In [148]:
val_mask

tensor([False, False, False,  ..., False, False, False])

In [149]:
test_mask

tensor([False, False, False,  ...,  True,  True,  True])

In [150]:
clf = LogisticRegression()
clf.fit(g.ndata['feat'][train_mask], label[train_mask])

#y_hat_val_lr = clf.predict_proba(X_kbest_features[val_mask,:])
y_val_lr = clf.predict(g.ndata['feat'][val_mask])

In [151]:
accuracy_score(label[val_mask], y_val_lr)

0.8911205073995772

In [152]:
clf_balanced = LogisticRegression(class_weight='balanced', max_iter=1000)
clf_balanced.fit(g.ndata['feat'][train_mask], label[train_mask])

y_val_lrb = clf_balanced.predict(g.ndata['feat'][val_mask])
accuracy_score(label[val_mask], y_val_lrb)

0.8911205073995772

In [153]:
rf = RandomForestClassifier(n_estimators=400)
rf.fit(g.ndata['feat'][train_mask,:], label[train_mask])

y_val_rf = rf.predict(g.ndata['feat'][val_mask,:])
accuracy_score(label[val_mask], y_val_rf)

0.8953488372093024

In [154]:
gb = GradientBoostingClassifier(n_estimators=300, learning_rate=1.0, max_depth=1, random_state=0)
gb.fit(g.ndata['feat'][train_mask,:], label[train_mask])

y_val_rf = gb.predict(g.ndata['feat'][val_mask,:])
accuracy_score(label[val_mask], y_val_rf)

0.8604651162790697

In [155]:
ab = AdaBoostClassifier(n_estimators=400, random_state=0)
ab.fit(g.ndata['feat'][train_mask,:], label[train_mask])

y_val_rf = ab.predict(g.ndata['feat'][val_mask,:])
accuracy_score(label[val_mask], y_val_rf)

0.8625792811839323

In [156]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
cv = KFold(n_splits=10, random_state=7, shuffle=True)

In [157]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(max_iter=10000) 
scores = cross_val_score(log, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.851797040169133


In [158]:
from sklearn.svm import LinearSVC
lsvm = LinearSVC()
scores = cross_val_score(lsvm, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8782241014799155


In [159]:
from sklearn.linear_model import PassiveAggressiveClassifier
pa=PassiveAggressiveClassifier(max_iter=1000, random_state=7, tol=1e-3)
scores = cross_val_score(pa, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8668076109936574


In [160]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=300)
scores = cross_val_score(rf, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8558139534883722


In [161]:
from sklearn.ensemble import AdaBoostClassifier
ab = AdaBoostClassifier(n_estimators=300)
scores = cross_val_score(ab, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8450317124735729


In [162]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=300)
scores = cross_val_score(gb, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8553911205073996


In [163]:
import xgboost as xgb
xgbc = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, 
              gamma=0,  importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, 
             n_estimators=200, n_jobs=16,
              num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, subsample=1,
              tree_method='exact', use_label_encoder=False
              )
scores = cross_val_score(xgbc, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.86553911205074


In [164]:
# Contruct a two-layer GNN model
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        self.conv1 = dglnn.SAGEConv(
            in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
        self.conv2 = dglnn.SAGEConv(
            in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')

    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

In [165]:
node_features = g.ndata['feat'].float()
node_labels = g.ndata['label']
train_mask = g.ndata['train_mask']
valid_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
n_features = node_features.shape[1]
n_labels = int(node_labels.max().item() + 1)

In [166]:
node_features

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0577, 0.0000, 0.0000,  ..., 0.0587, 0.0549, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0504, 0.0000]])

In [167]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [168]:
model = SAGE(in_feats=n_features, hid_feats=100, out_feats=n_labels)
opt = torch.optim.Adam(model.parameters())

for epoch in range(50):
    model.train()
    # forward propagation by using all nodes
    logits = model(g, node_features)
    # compute loss
    loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
    # compute validation accuracy
    acc = evaluate(model, g, node_features, node_labels, test_mask)
    print(acc)
    # backward propagation
    opt.zero_grad()
    loss.backward()
    opt.step()
    #print(loss.item())

    # Save model if necessary.  Omitted in this example.

0.4334038054968288


/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


0.5169133192389006
0.6342494714587738
0.7241014799154334
0.806553911205074
0.8583509513742071
0.8847780126849895
0.8932346723044398
0.9038054968287527
0.919661733615222
0.9302325581395349
0.9344608879492601
0.9408033826638478
0.9418604651162791
0.9418604651162791
0.9439746300211417
0.9471458773784355
0.9482029598308668
0.9482029598308668
0.9503171247357294
0.9534883720930233
0.9534883720930233
0.9566596194503171
0.959830866807611
0.9608879492600423
0.9608879492600423
0.9619450317124736
0.9630021141649049
0.9640591966173362
0.9651162790697675
0.9661733615221987
0.9661733615221987
0.96723044397463
0.96723044397463
0.9682875264270613
0.9704016913319239
0.9704016913319239
0.9704016913319239
0.9704016913319239
0.9704016913319239
0.9704016913319239
0.9704016913319239
0.9704016913319239
0.9704016913319239
0.9704016913319239
0.9704016913319239
0.9704016913319239
0.9704016913319239
0.9704016913319239
0.9704016913319239
